# chapter 4 - Web Mining Techniques
웹 마이닝 기법

<br></br>
## 1 웹 구조 마이닝
Web structure mining

### 01 웹 크롤러 - Spyder
### 02 인덱서- DB
### 03 페이지 랭킹 알고리즘 - Ranking (명성)
page I's in-link : 외부 페이지에서 I page 연결 수

page I's out-link : 페이지 I 가 연결하는 외부페이지 수
1. P(i) : i 페이지 방문확률
2. P(j) : j 페이지 방문확률
3. Aji: 노트 j에서 노드 i로 전이될 확률 
$$ P(i) = \sum_j A_{ji} P(j) $$

### 04 rank sing (랭크싱크) 
서로 링크한 경우에는 Loop에 빠져서, 랜덤한 점프 전이행렬을 추가
1. (1-d) 페이지를 임의로 방문할 확률
2. A는 확룰로써 총 합은 1이다
$$ P = (\frac{(1-d)E}{N}+dA^T)P $$
3. 아래의 식의 P벡터를 $e^T P=N $으로 정규화를 하면 식이 간단해진다
$$ P = (1-d)e + dA^TP \to P(i) = (1-d)+d\sum_{j=1}A_{ji}P(j) $$

<br></br>
## 2 웹 콘텐츠 마이닝
### 01 Parsing
파싱
### 02 자연어 처리
NLTK

Stemming : 단어를 어간으로 축소

In [1]:
text = """On 1 January 2007, Irish became a full EU official language, 
with a temporary derogation for a renewable period of five years (see Council 
Regulation (EC) No 920/2005 of 13 June 2005 (OJ L 156, 18.6.2005, p. 3)) stating 
that 'the institutions of the European Union shall not be bound by the obligation
to draft all acts in Irish and to publish them in that language in the Official 
Journal of the European Union """

In [2]:
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()

words = tknzr.tokenize(text)
print('Words length : ', len(words), '\n\n', words[:20])

Words length :  91 

 ['On', '1', 'January', '2007', ',', 'Irish', 'became', 'a', 'full', 'EU', 'official', 'language', ',', 'with', 'a', 'temporary', 'derogation', 'for', 'a', 'renewable']


In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

words_clean = [ w.lower()   for w in words   if w not in stopwords]
print('Clean words length : ', len(words_clean), '\n\n', words_clean[:15])

Clean words length :  64 

 ['on', '1', 'january', '2007', ',', 'irish', 'became', 'full', 'eu', 'official', 'language', ',', 'temporary', 'derogation', 'renewable']


In [4]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

words_clean_stem = [stemmer.stem(w)  for w in words_clean]
print('Clean words stem length : ', len(words_clean_stem), '\n\n', words_clean_stem[:15])
# token 의 갯수는 동일, 접사를 제거한 나머지를 출력

Clean words stem length :  64 

 ['on', '1', 'januari', '2007', ',', 'irish', 'becam', 'full', 'eu', 'offici', 'languag', ',', 'temporari', 'derog', 'renew']


<br></br>
## 3 정보 검색 모델
Information retrieval models

모델목록 : Boolean Model(불리언), Vector space model(벡터), Probabilistic model(확률)

Vector 모델 목록
1. TF-IDF(Term Frequency-Inverse Document Frequency) 단어빈도 - 역문서 빈도
2. LSA(Latent Semantic Analysis) 잠재의미분석
3. Doc2Vec (Word2Vec)

쿼리 단어간의 벡터로 표현하고, <strong>쿼리 벡터</strong>와 <strong>각 문서</strong>간의 <strong>$\cos \theta$ 유사도</strong> 를 측정, 비교한다 

### 01 TF-IDF
아주 <strong>많은 문서에 빈번</strong>하게 등장하는 단어는 <strong>중요도가 낮고</strong>, 
<strong>일부문서에 빈번</strong>한 단어의 <strong>중요도가 높다</strong>
$$ W_{ij} = tf_{ij}*idf_j $$
$ tf_{ij} = \frac {f_{ij}}{max f_{i1}...f_{iV}}$  문서 i에서 <strong>단어 j의 정규화 빈도</strong>

$ idf_j = log\frac{N}{df} $ <strong>역문서 빈도</strong>로 $idf_j$는 단어 j를 포함하는 웹페이지 갯수 (N은 전체 웹페이지 갯수)

### 02 잠재 의미 분석(LSA)
단어와 문서를 효과적으로 설명할 수 있는 정형적인 잠재공간이 존재하고, 비슷한 의미의 단어는 비슷한 위치에 나타난다

문서의 잠재공간 투영은 <strong>절단SVD</strong> (Singular Value Decomposition)를 이용한다
$$ X = U_t \sum_t V_t^T $$
$U_t(V*t)$ : t 차원의 잠재공간에 투영된 단어의 행렬

$\sum_t V_t^T $ : 잠재공간에 투영된 문서의 전치행렬

$\sum_{t(t*t)}$ : 특기앖으로 이루어진 대각행렬
$$ q_t = q^TU_t\sum_t^{-1}$$ 쿼리벡터도 잠재공간에 투영된다

각 문서는 <strong>$q_t^T$</strong>와 <strong>$\cos\theta$유사도</strong>로 비교될 수 있지만, 

실제는 $V_t^T$와 $q_t^T$간의 <strong>유사도</strong>로 계산한다

### 03 Word2Vec
두가지 아키텍처 알고리즘 중 하나를 선택하려, N개의 뉴런(Neuron)(가중치)으로 된 Hidden Layer를 훈련시킨다.

즉 N개의 뉴런(가중치)h를 갖는 단일계층 학습으로 얕은학습(shallow learning)방식이다 

행렬W는 입력벡터를 hidden layer로 변환하며, 출력계층에서는 목표를 평가한다

train단계에서 W와 W"를 수정하기 위해서 확률적 내리막 경사법(Stochastic Gradient Descent)를 사용한다

https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/

<h4><strong>연속단어 주머니 CBOW (Continuous Bag Of Words)</strong></h4>
<h5>ex) 편의점에서 아이스크림을 사 먹었는데, ___ 시려서 힘들었다. (의 빈칸에 알맞은 단어찾기)</h5>
주어진 단어 앞 뒤 C/2개 씩 <strong>총 C개 단어를 Input</strong>으로 사용, 

<strong>주어진 단어를 맞추기 위한 네트워크</strong>를 만든다
<h4>Input Layer 는</h4> 모든 단어들이 공통적으로 사용하는 VxN 
(N:사용벡터의 길이(Projection Layer길이)) 크기의 Projection Matrix $W$가 있고
<h4>Projection Layer에서 Output Layer로 갈 때는</h4> NxV 크기의 Weight Matrix W’ 가 있다.  
(주의해야할 점은, 두 행렬은 transpose이 아닌, 별개의 행렬이라는 점이다)
<h4>Process 정리</h4>
1. 즉 단어들을 one-hot encoding으로 입력
2. 여러단어를 각각 projection 시킨 후, 벡터들의 평균을 구해서 Projection Layer에 보낸다. 
3. Weight Matrix를 곱해서 Output Layer로 보내고 softmax 계산을 한 후, 
4. 이 결과를 진짜 단어의 one-hot encoding과 비교하여 에러를 계산한다.

<h4>CBOW 모델에서 하나의 단어 처리 계산량</h4>
1. C개의 단어를 Projection 하는 데에 C x N
2. Projection Layer에서 Output Layer로 가는 데에 N x V
전체 계산량은 CxN + NxV가 되는데, V를 ln V로 줄이는 테크닉을 사용하면 

전체 계산량이 CxN + N x lnV가 된다.

결국 <strong>Projection Layer의 크기 N</strong>과 <strong>log-사전크기의 lnV</strong>의 크기의 곱에 비례하게 된다.

C=10, N=500, V=1,000,000으로 잡아도 500 x (10+ln(1,000,000)) = 약 10000의 계산량밖에 들지 않는 것이다. 

이는 앞서 확인한 NNLM이나 RNNLM에 비해 정말 엄청나게 줄어든 계산량이라는 것을 확인할 수 있다.

<img src="https://shuuki4.files.wordpress.com/2016/01/cbow.png?w=260&h=300" align="left">

<h4><strong>스킵그램(Skip-gram)</strong></h4>

주어진 단어의, 주위에 등장하는 나머지 몇 가지의 단어들의 등장 여부를 유추하는 것으로

‘가까이 위치한 단어일 수록 관련이 더 많다’ 라는 생각을 적용하기 위해 


멀리 떨어져있는 단어일수록 낮은 확률로 택하는 방법을 사용한다. 

나머지 구조는 CBOW와 방향만 반대일 뿐 굉장히 유사하다

<h4>Skip-gram에서 하나의 단어를 처리하는 데에 드는 계산량</h4>

1. C개의 단어를 샘플링했다고 할 때, 현재 단어를 Projection 하는 데에 N
2. Output을 계산하는 데에 N x V, 테크닉을 사용하면 N x ln V
3. 총 C개의 단어에 대해 진행해야 하므로 총 C배
4. 총 C(N + N x lnV) 만큼의 연산이 필요하다. 

CBOW 모델같이 N x lnV에 비례하는 계산량을 가진 모델이기는 하지만, 

샘플링 단어 갯수에 따라서 계산량이 비례하여 올라가게 되므로 CBOW 비해서는 느리다

실험 결과에서는 Skip-gram이 CBOW에 비해 전체적으로 다소 좋은 결과를 내는 추세를 보인다

<img src="https://shuuki4.files.wordpress.com/2016/01/skip-gram.png?w=276&h=300" align="left">

### 04 Doc2Vec
단어 $W_j$를 벡터 $V_w$로 표현하며, 단어가 나타나는 문서 $d_i$와는 독립적 벡터로 정의한다

Word2Vec알고리즘을 <strong>신경망</strong>과 <strong>역전파</strong>를 확장 이용한 것으로

문서를 단어의 벡터로 추가한 모델이다
<h4><strong>분산 메모리 모델 (DM) Distributed Memory model</strong>로써</h4>
문서 <strong>D</strong>벡터로 수집한 단어들이 학습모델과 공유되며, 

W와 W"는 모든 문서에 공통으로 연산하게 되어서 최적값을 찾는다 
<h4><strong>분산 단어 주머니(DBOW) distributed bag of words</strong> </h4>
https://radimrehurek.com/gensim/models/doc2vec.html

입력계층에서는 문서 벡터만 고려하고, 출력계층은 문서 샘플링 Context의 단어 집합만 고려한다


DM아키텍처는 Gensim 라이브러리에 기본모듈로써 구현되어 있다
<h5>(아래는 3개 단어로 된 컨텍스트를 찾는 분산 메모리 모델의 예제이다)</h5>
<img src="https://research.google.com/archive/paragraph_vector_icml2014/distributed_memory_model.png" align="left" width="600">

<br></br>
## 4 영화 리뷰 쿼리분석 예제

### 01 데이터 불러오기
bs4 옵션설정 https://www.pythonanywhere.com/forums/topic/7002/  

open() utf-8 오류설정 https://stackoverflow.com/questions/20578270/unicodedecodeerror-in-python-while-reading-utf-8-sql-file-from-english-wikipedia  


In [73]:
import os
from time import time
import numpy as np
from bs4 import BeautifulSoup

movie_html_dir = './data/movie/'
movie_dict = {} # 데이터 파일 목록 저장

In [81]:
filenames = [f    for f in os.listdir(movie_html_dir)      if f[0]!='.'] ; t0 = time()
for file in filenames:
    id_html = file.split('.')[0]
    # UTF-8 오류발생시 외부 메세지 처리없이 결과를 출력한다
    f = open(movie_html_dir + file, encoding = "utf8", errors = 'replace')
    s = f.read()    
    parsed_html = BeautifulSoup(s,"lxml")
    try:     title = parsed_html.body.h1.text
    except:  title = 'none'
    movie_dict[id_html] = title
print('Web page :', len(filenames), '\nCrawling time :',round(time()-t0, 4))

Web page : 27886 
Crawling time : 37.3517


### 02 긍/부정 Text 사전처리
nltk_movie_review

In [85]:
# 모듈호출 및 불용어 목록 불러오기
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

tknzr = WordPunctTokenizer()
nltk.download('stopwords')
stoplist = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/markbaum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 어간처리 모듈 불러오기
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def ListDocs(dirname):
    docs, titles  = [], []
    filenames = [f for f in os.listdir(dirname) if str(f)[0]!='.']
    for filename in filenames:
        f = open(dirname+'/'+filename,'r')
        id_html = filename.split('.')[0].split('_')[1]
        titles.append(movie_dict[id_html])
        docs.append(f.read())
    return docs, titles

In [94]:
# 리뷰 텍스트 모두 메모리에 불러오기
dir = '/home/markbaum/nltk_data/corpora/movie_reviews/'
pos_textreviews, pos_titles = ListDocs(dir+'pos/')
neg_textreviews, neg_titles = ListDocs(dir+'neg/')
tot_textreviews = pos_textreviews + neg_textreviews
tot_titles = pos_titles + neg_titles

neg_textreviews[2][:500]

'david schwimmer ( from the television series " friends " ) stars as a sensitive ( and slightly neurotic ) single guy who gets more than he expected from the grieving mother ( barbara hershey ) of a classmate he can\'t remember . \nhello mrs . robinson ! \nthough quite cute as a romantic comedy , the pallbearer is paced like a funeral march . \nthe characters act , react , and interact at half-speed , making for one * excruciatingly * long sit . \n ( and what\'s with the dreary lighting ? ) \nco-writer/'

In [ ]:
# 저장 데이터 목록 
# pos_titles
# neg_titles
# tot_titles

# pos_textreviews
# neg_textreviews
# tot_textreviews

### 03 TF-IDF 구현하기 sklearn 모듈 활용
TF-IDF model can be trained using sklearn

In [96]:
# Text 전처리 함수
def Preprocess_Tf_idf(texts, stoplist = [], stem = False):
    newtexts = []
    for text in texts:
        # 불용어 제거하기
        if stem:   tmp = [w for w in tknzr.tokenize(text) if w not in stoplist]
        # 유효한 단어 어간 처리 (접사 제거)
        else:      tmp = [stemmer.stem(w) for w in [w for w in tknzr.tokenize(text) if w not in stoplist]]
        newtexts.append(' '.join(tmp))
    return newtexts

# neg/pos 영화리뷰 데이터 전처리 하기
processed_reviews = Preprocess_Tf_idf(tot_textreviews, stoplist, True)

In [115]:
# tf-idf 모델 테스트 하기
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 1)            # tf-idf 모듈 활성화
mod_tfidf = vectorizer.fit(processed_reviews)       # tf-idf 데이터 적용
vec_tfidf = mod_tfidf.transform(processed_reviews)  # tf-idf 벡터 변환
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

print(mod_tfidf); i = 0
# tfidf 살펴보기
for item, value in tfidf.items():
    print("{",item,":",value,"}"); i += 1
    if i > 5 : break

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
{ 00 : 6.40417775725 }
{ 000 : 4.06880284143 }
{ 0009f : 7.90825515402 }
{ 007 : 6.29881724159 }
{ 00s : 7.90825515402 }
{ 03 : 7.90825515402 }


In [132]:
v.shape

(2000, 39516)

In [ ]:
import  pickle  # tf-idf 객체 저장하기
print ('학습한 html 리뷰 수:',len(processed_reviews),'--> TF-IDF 저장객체 :', len(mod_tfidf.get_feature_names()))
print ('TF-IDF :', mod_tfidf.get_feature_names()[::4000])
v = mod_tfidf.transform( processed_reviews )
v.shape

In [123]:
# 학습한 객체를 저장한다
with open('vectorizer.pk', 'wb') as file:
      pickle.dump(mod_tfidf, file)

# 저장한 객체를 불러온다
load_tfidf = pickle.load( open("vectorizer.pk", 'rb') )
load_tfidf.transform(PreprocessTfidf([' '.join(['drama'])],stoplist,True))

2000 -- 39516


<1x39516 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

### 04 gensim 의 LSA 모델 적용
단어 쿼리를 잠재공간으로 변환시 필요한 U,V,S 행렬을 얻는다

In [ ]:
import gensim
from gensim import models
class GenSimCorpus(object):
    def __init__(self, texts, stoplist=[],stem=False):
        self.texts = texts
        self.stoplist = stoplist
        self.stem = stem
        self.dictionary = gensim.corpora.Dictionary(self.iter_docs(texts, stoplist))
    def __len__(self):  return len(self.texts)
    def __iter__(self):
        for tokens in self.iter_docs(self.texts, self.stoplist):
            yield self.dictionary.doc2bow(tokens)
    def iter_docs(self,texts, stoplist):
        for text in texts:
            if self.stem: yield (stemmer.stem(w) for w in [x for x in tknzr.tokenize(text) if x not in stoplist])
            else:         yield (x for x in tknzr.tokenize(text) if x not in stoplist)

In [146]:
# corpus 생성하기
corpus = GenSimCorpus(tot_textreviews, stoplist, True)
dict_corpus = corpus.dictionary
print('corpus     :', corpus); print('dict_corpus:', dict_corpus)

corpus     : <__main__.GenSimCorpus object at 0x7f85f9f7e3c8>
dict_corpus: Dictionary(26132 unique tokens: ['film', 'unexpect', ',', 'scari', 'origin']...)


In [136]:
# U matrix 생성
lsi =  models.LsiModel(corpus, num_topics = ntopics, id2word = dict_corpus)
U = lsi.projection.u
U.shape

(26132, 10)

In [138]:
# Sigma 생성
ntopics = 10
Sigma = np.eye(ntopics) * lsi.projection.s
Sigma.shape

(10, 10)

In [140]:
# V 행렬을 계산한다
V = gensim.matutils.corpus2dense(lsi[corpus], len(lsi.projection.s)).T / lsi.projection.s
V.shape 

(2000, 10)

In [156]:
# 쿼리에 포함된 단어의 색인을 생성한다
dict_words = {}
for i in range(len(dict_corpus)):
    dict_words[dict_corpus[i]] = i
    
# http://hamait.tistory.com/803  # 반복 실행문의 부분실행 모듈
import itertools as it
print('dict_word\'s length' : , len(dict_words))
{k: dict_words[k] for k in it.islice(dict_words.keys(), 10) }

dict_word's length 26132


{',': 2,
 '.': 9,
 'caught': 5,
 'film': 0,
 'guard': 6,
 'loop': 8,
 'origin': 4,
 'scari': 3,
 'threw': 7,
 'unexpect': 1}

In [ ]:
from itertools import islice

for i in islice(range(10), 5):
    print i




{',': 2,
 '.': 9,
 'caught': 5,
 'film': 0,
 'guard': 6,
 'loop': 8,
 'origin': 4,
 'scari': 3,
 'threw': 7,
 'unexpect': 1}

In [ ]:
it.islice